In [6]:
import traci

# Use sumo or sumo-gui depending on your preference
sumo_binary = "sumo"  # or "sumo-gui"
sumo_config = "./sumo_cfg/simulation.sumocfg"  # your .sumocfg file

traci.start([sumo_binary, "-c", sumo_config])

# Run simulation for 100 steps
for step in range(100):
    traci.simulationStep()

    # Get traffic light IDs
    tls_ids = traci.trafficlight.getIDList()
    for tls_id in tls_ids:
        phase = traci.trafficlight.getPhase(tls_id)
        print(f"Step {step}: Traffic light {tls_id} is at phase {phase}")

traci.close()


 Retrying in 1 seconds


Step 0: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 1: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 2: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 3: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 4: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 5: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 6: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 7: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 8: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 9: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 10: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 11: Traffic light cluster_13075564400_13075589603_411926344 is at phase 0
Step 12: Traffic light cluster_13075564400_13075589603_4119263